In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Eur_Usd_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Eur_Usd_M1_2021-2022.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   1.17170   1.17172  1.17141    1.17141   1.17184   
1 2021-04-01 06:05:00   1.17140   1.17182  1.17119    1.17154   1.17153   
2 2021-04-01 06:10:00   1.17153   1.17162  1.17140    1.17161   1.17166   
3 2021-04-01 06:15:00   1.17160   1.17187  1.17153    1.17185   1.17173   
4 2021-04-01 06:20:00   1.17184   1.17253  1.17171    1.17232   1.17196   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.17187  1.17154    1.17154   1.17177   1.17180  1.17148    1.17148   
1   1.17194  1.17133    1.17167   1.17146   1.17188  1.17126    1.17160   
2   1.17175  1.17151    1.17174   1.17160   1.17168  1.17146    1.17168   
3   1.17200  1.17165    1.17197   1.17166   1.17194  1.17160    1.17191   
4   1.17266  1.17185    1.17244   1.17190   1.17259  1.17178    1.17238   

   Volume  
0     348  
1     451  
2     324  
3     275  
4     646  
--------------------------

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   1.17170   1.17172  1.17144    1.17148   1.17184   
1 2021-04-01 06:01:00   1.17148   1.17157  1.17146    1.17155   1.17159   
2 2021-04-01 06:02:00   1.17154   1.17158  1.17141    1.17146   1.17167   
3 2021-04-01 06:03:00   1.17145   1.17162  1.17141    1.17160   1.17157   
4 2021-04-01 06:04:00   1.17160   1.17160  1.17141    1.17141   1.17171   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.17187  1.17157    1.17161   1.17177   1.17180  1.17150    1.17154   
1   1.17169  1.17159    1.17168   1.17154   1.17163  1.17152    1.17162   
2   1.17170  1.17154    1.17158   1.17160   1.17164  1.17148    1.17152   
3   1.17174  1.17154    1.17173   1.17151   1.17168  1.17148    1.17166   
4   1.17171  1.17154    1.17154   1.17166   1.17166  1.17148    1.17148   

   Volume  
0     100  
1      60  
2      73  
3      72  
4      43  
--------------------------

In [6]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan


In [7]:
# Add technical indicators (for additional features)
df['atr'] = atr(df)
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-04-01 07:10:00   1.17239   1.17334  1.17239    1.17316   1.17252   
1     2021-04-01 07:15:00   1.17315   1.17321  1.17274    1.17297   1.17329   
2     2021-04-01 07:20:00   1.17298   1.17362  1.17263    1.17362   1.17311   
3     2021-04-01 07:25:00   1.17364   1.17420  1.17352    1.17414   1.17377   
4     2021-04-01 07:30:00   1.17412   1.17425  1.17381    1.17394   1.17425   
...                   ...       ...       ...      ...        ...       ...   
74764 2022-04-01 05:35:00   1.10690   1.10694  1.10668    1.10676   1.10707   
74765 2022-04-01 05:40:00   1.10675   1.10727  1.10672    1.10720   1.10691   
74766 2022-04-01 05:45:00   1.10721   1.10722  1.10698    1.10704   1.10737   
74767 2022-04-01 05:50:00   1.10703   1.10729  1.10700    1.10726   1.10721   
74768 2022-04-01 05:55:00   1.10727   1.10737  1.10667    1.10673   1.10742   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_

In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
pair = 'EUR_USD'

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(open_price, stop_loss, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = abs(open_price - stop_loss)
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, atr_multiplier, n_minutes, adx_cutoff, use_ema, invert, use_trailing_sl):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    day_fees = 0
    end_date = None
    limit_order = None

    def iterate_thru_small_data(lo):
        def get_end_date(j):
            return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        filtered = df_small[df_small.Date >= lo['start_date']]

        if len(filtered) == 0:
            return -1

        j = filtered.index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal n_buys
        nonlocal n_sells
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak
        nonlocal pips_risked
        nonlocal win_amounts
        nonlocal loss_amounts
        trade = None

        while j < len(df_small):
            curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, cur_date = df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Date']]

            if lo is not None:
                minutes_elapsed = (cur_date - lo['start_date']).total_seconds() / 60
                curr_long = df[df.Date < cur_date]
                adx_long, ema200_long, mid_close_long = curr_long.loc[curr_long.index[-1], ['adx', 'ema200', 'Mid_Close']]

                if minutes_elapsed >= n_minutes or adx_long >= adx_cutoff:
                    return cur_date

            if trade is None and lo is not None:
                low = curr_bid_low if invert else curr_ask_low
                high = curr_ask_high if invert else curr_bid_high
                bottom_hit = 'buy_open_price' in lo and low <= lo['buy_open_price']
                top_hit = 'sell_open_price' in lo and high >= lo['sell_open_price']

                if bottom_hit:
                    trade_type = 'buy' if not invert else 'sell'

                    open_price = lo['buy_open_price']
                    stop_gain = lo['buy_stop_gain']
                    pips_gained = stop_gain - open_price
                    curr_pips_to_risk = pips_gained / risk_reward_ratio
                    stop_loss = lo['buy_stop_loss']

                    if invert:
                        stop_loss = open_price + curr_pips_to_risk
                        stop_gain = open_price - pips_gained

                    n_units = lo['buy_n_units']

                    trade = {'open_price': open_price, 'trade_type': trade_type, 'stop_loss': stop_loss,
                                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                    'n_units': n_units, 'original_units': n_units, 'start_date': cur_date, 'end_date': None, 'prev_profit_ratio': None}

                    n_buys += 1 if not invert else 0
                    n_sells += 1 if invert else 0

                    pips_risked.append(curr_pips_to_risk)
                    
                    lo = None

                elif top_hit:
                    trade_type = 'sell' if not invert else 'buy'

                    open_price = lo['sell_open_price']
                    stop_gain = lo['sell_stop_gain']
                    pips_gained = open_price - stop_gain
                    curr_pips_to_risk = pips_gained / risk_reward_ratio
                    stop_loss = lo['sell_stop_loss']

                    if invert:
                        stop_loss = open_price - curr_pips_to_risk
                        stop_gain = open_price + pips_gained

                    n_units = lo['sell_n_units']

                    trade = {'open_price': open_price, 'trade_type': trade_type, 'stop_loss': stop_loss,
                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                            'n_units': n_units, 'original_units': n_units, 'start_date': cur_date, 'end_date': None, 'prev_profit_ratio': None}

                    n_sells += 1 if not invert else 0
                    n_buys += 1 if invert else 0

                    pips_risked.append(curr_pips_to_risk)

                    lo = None

            if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None
                break

            if trade is not None and trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
                curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

                # Initial move
                if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                    trade['stop_loss'] = trade['open_price']
                    trade['prev_profit_ratio'] = 0.0

                # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
                #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
                #     trade['prev_profit_ratio'] = 0.5

                # Subsequent moves
                if curr_profit_ratio >= 2.0:
                    # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                    while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                        # trade['prev_profit_ratio'] += 0.5
                        trade['prev_profit_ratio'] += 1.0
                        trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

            if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
                trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None
                break

            if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None

            if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
                curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

                # Initial move
                if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                    trade['stop_loss'] = trade['open_price']
                    trade['prev_profit_ratio'] = 0.0

                # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
                #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
                #     trade['prev_profit_ratio'] = 0.5

                # Subsequent moves
                if curr_profit_ratio >= 2.0:
                    # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                    while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                        # trade['prev_profit_ratio'] += 0.5
                        trade['prev_profit_ratio'] += 1.0
                        trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

            if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
                trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount > 0:
                    win_amounts.append(trade_amount)

                else:
                    loss_amounts.append(trade_amount)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None
                break

            j += 1

        return cur_date

    for i in range(1, len(df)):
        if end_date == -1:
            break

        curr_date = df.loc[df.index[i], 'Date']

        if end_date is not None and end_date >= curr_date:
            continue

        end_date = None

        curr_date, mid_high, mid_low, mid_close, ask_close, bid_close, adx, atr, ema200 = df.loc[df.index[i - 1], ['Date', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Ask_Close', 'Bid_Close', 'adx', 'atr', 'ema200']]
        spread = abs(ask_close - bid_close)
        chop_signal = adx < adx_cutoff

        if limit_order is None and chop_signal:
            range_high, range_low = mid_high + (atr * atr_multiplier), mid_low - (atr * atr_multiplier)
            rang = abs(float(range_high) - float(range_low))
            pips_gain = (rang / 2) * risk_reward_ratio
            pips_to_risk = pips_gain / risk_reward_ratio
            
            if range_high > range_low and spread <= pips_to_risk * spread_cutoff:
                buy_open_price = float(range_low)
                buy_stop_gain = buy_open_price + pips_gain
                buy_pullback = buy_open_price - pips_to_risk
                buy_stop_loss = round(buy_pullback, 5)
                buy_mo = buy_open_price - (spread / 2)
                buy_n_units = get_n_units(buy_open_price, buy_stop_loss, buy_mo, pair)

                sell_open_price = float(range_high)
                sell_stop_gain = sell_open_price - pips_gain
                sell_pullback = sell_open_price + pips_to_risk
                sell_stop_loss = round(sell_pullback, 5)
                sell_mo = sell_open_price + (spread / 2)
                sell_n_units = get_n_units(sell_open_price, sell_stop_loss, sell_mo, pair)

                if use_ema:
                    if mid_close > ema200:
                        limit_order = {'buy_open_price': buy_open_price, 'buy_stop_gain': buy_stop_gain, 'buy_stop_loss': buy_stop_loss, 'buy_n_units': buy_n_units, 'start_date': curr_date}

                    else:
                        limit_order = {'sell_open_price': sell_open_price, 'sell_stop_gain': sell_stop_gain, 'sell_stop_loss': sell_stop_loss, 'sell_n_units': sell_n_units, 'start_date': curr_date}

                else:
                    limit_order = {'buy_open_price': buy_open_price, 'buy_stop_gain': buy_stop_gain, 'buy_stop_loss': buy_stop_loss, 'buy_n_units': buy_n_units, 
                               'sell_open_price': sell_open_price, 'sell_stop_gain': sell_stop_gain, 'sell_stop_loss': sell_stop_loss, 'sell_n_units': sell_n_units, 'start_date': curr_date}

        if limit_order is not None:
            end_date = iterate_thru_small_data(limit_order)
            limit_order = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0]
spread_cutoffs = [0.1, 0.2]
atr_multipliers = [0.5, 1.0]  # Determines how wide to make the sideways range
n_minutes_vals = [15, 20]
adx_cutoffs = [25, 30]
use_ema_vals = [True, False]
invert_vals = [True, False]
use_trailing_sl_vals = [True, False]


# Best reward: 2051.6826121429153
# Best risk/reward ratio: 1.0
# Best spread: 0.2
# Best atr multiplier: 0.5
# Best n minutes: 15
# Best adx cutoff: 25
# Best use ema: False
# Best invert: True
# Best use trailing sl val: False

all_combos = []

for spread_val in spread_cutoffs:
    for atr_multiplier in atr_multipliers:
        for n_minutes in n_minutes_vals:
            for adx_cutoff in adx_cutoffs:
                for use_ema in use_ema_vals:
                    for invert in invert_vals:
                        for use_trailing_sl in use_trailing_sl_vals:
                            for risk_reward_ratio in risk_reward_ratio_vals:
                                all_combos.append((risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff, use_ema, invert, use_trailing_sl))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_atr_multiplier = None
best_n_minutes= None
best_adx_cutoff = None
best_use_ema_val = None
best_invert_val = None
best_use_trailing_sl_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff, use_ema, invert, use_trailing_sl in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff, use_ema, invert, use_trailing_sl)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'atr_multiplier': atr_multiplier, 'n_minutes': n_minutes, 'adx_cutoff': adx_cutoff, 'use_ema': use_ema, 'invert': invert, 'use_trailing_sl': use_trailing_sl})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_atr_multiplier = atr_multiplier
        best_n_minutes = n_minutes
        best_adx_cutoff = adx_cutoff
        best_use_ema_val = use_ema
        best_invert_val = invert
        best_use_trailing_sl_val = use_trailing_sl

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 128

50.132680000008136
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 1
Num losses: 1
Win streak: 1
Loss streak: 1
Avg pips risked: 0.0013542857142857523
Avg win amount: 100.10644000000401
Min win amount: 100.10644000000401
Max win amount: 100.10644000000401
Avg loss amount: -49.97375999999588
Min loss amount: -49.97375999999588
Max loss amount: -49.97375999999588
Remaining runs: 127
Best reward so far: 50.132680000008136

150.47636000000523
Num buys: 2
Num sells: 1
Num trades: 3
Num wins: 2
Num losses: 1
Win streak: 2
Loss streak: 1
Avg pips risked: 0.0016840476190475389
Avg win amount: 100.22506000000055
Min win amount: 49.957399999998366
Max win amount: 150.49272000000275
Avg loss amount: -49.97375999999588
Min loss amount: -49.97375999999588
Max loss amount: -49.97375999999588
Remaining runs: 126
Best reward so far: 150.47636000000523

0.18134285714279486
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 1
Num losses: 1
Win streak: 1
Loss streak: 1
Avg pips risked: 0.00

In [15]:
# EUR/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best use ema: ' + str(best_use_ema_val))
print('Best invert: ' + str(best_invert_val))
print('Best use trailing sl val: ' + str(best_use_trailing_sl_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1448.774479285797
Best risk/reward ratio: 1.0
Best spread: 0.2
Best atr multiplier: 1.0
Best n minutes: 20
Best adx cutoff: 25
Best use ema: False
Best invert: True
Best use trailing sl val: False
-----------------------
Top results:
{'reward': 648, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 0.5, 'n_minutes': 15, 'adx_cutoff': 30, 'use_ema': False, 'invert': True, 'use_trailing_sl': False}
{'reward': 1348, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 30, 'use_ema': False, 'invert': True, 'use_trailing_sl': False}
{'reward': 998, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25, 'use_ema': False, 'invert': True, 'use_trailing_sl': False}
{'reward': 676, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 10, 'adx_cutoff': 30, 'use_ema': False, 'invert': True, 'use_trailing_sl': False}
{'reward': 699, 'ratio': 0.25, 'spread_val':

In [14]:
# GBP/CHF
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best use ema: ' + str(best_use_ema_val))
print('Best invert: ' + str(best_invert_val))
print('Best use trailing sl val: ' + str(best_use_trailing_sl_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 399.5970799999996
Best risk/reward ratio: 1.0
Best spread: 0.1
Best atr multiplier: 1.0
Best n minutes: 20
Best adx cutoff: 30
Best use ema: True
Best invert: False
Best use trailing sl val: True
-----------------------
Top results:
{'reward': 150, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25, 'use_ema': True, 'invert': False, 'use_trailing_sl': False}
{'reward': 100, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 0.5, 'n_minutes': 15, 'adx_cutoff': 25, 'use_ema': True, 'invert': False, 'use_trailing_sl': False}
{'reward': 50, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 30, 'use_ema': False, 'invert': True, 'use_trailing_sl': False}
{'reward': 349, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25, 'use_ema': True, 'invert': False, 'use_trailing_sl': True}
{'reward': 50, 'ratio': 1.0, 'spread_val': 0.2, 